In [1]:
import os
par_dir = os.path.abspath(os.path.join(os.getcwd(),os.pardir))
os.chdir(par_dir)

In [2]:
import numpy as np
import pandas as pd
import joblib
x_data = np.load('./data/metal.npy')
c_data = np.load('./data/pre_re_opt.npy')
# s_data = np.load('./data/support.npy')
### train_test_split을 활용해 데이터를 train,val,test로 해서 0.6,0.2,0.2로 나눠서 구분
# - random_state = 21로 설정
from sklearn.model_selection import train_test_split
x_train,x_test,c_train,c_test = train_test_split(x_data,c_data, random_state = 22,test_size = 0.4)
x_val,x_test,c_val,c_test = train_test_split(x_test,c_test,random_state = 22, test_size = 0.5)

from sklearn.preprocessing import MinMaxScaler
x_scaler = MinMaxScaler()
c_scaler = MinMaxScaler()
# s_scaler = StandardScaler()
x_train = x_scaler.fit_transform(x_train)
c_train = c_scaler.fit_transform(c_train)
x_val,x_test = [x_scaler.transform(x) for x in [x_val,x_test]]
c_val,c_test = [c_scaler.transform(c) for c in [c_val,c_test]]
# import joblib 
joblib.dump(x_scaler,'./torch_1/min_x_scaler_opt.pkl')
joblib.dump(c_scaler,'./torch_1/min_pre_re_scaler_opt.pkl')

['./torch_1/min_pre_re_scaler_opt.pkl']

In [3]:
import torch
x_train,x_val,x_test = [torch.tensor(x, dtype = torch.float32) for x in [x_train,x_val,x_test]]
c_train,c_val,c_test = [torch.tensor(c, dtype = torch.float32) for c in [c_train,c_val,c_test]]
from torch.utils.data import TensorDataset,DataLoader
train_data = [x_train,x_train, c_train]
val_data = [x_val,x_val, c_val]
test_data = [x_test,x_test, c_test]
train_data = TensorDataset(*train_data)
val_data = TensorDataset(*val_data)
test_data = TensorDataset(*test_data)
datas = [train_data,val_data,test_data]
train_loader,val_loader,test_loader = [DataLoader(x,batch_size = 64,shuffle=False) for x in datas]

In [4]:
import os
os.makedirs('torch',exist_ok = True)
torch.save(train_loader,"torch_1/pre_retrain_loader_opt.pt")
torch.save(val_loader,"torch_1/pre_reval_loader_opt.pt")
torch.save(test_loader,"torch_1/pre_retest_loader_opt.pt")